In [3]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.spatial import Delaunay
from haversine import haversine, Unit
from datetime import datetime, timedelta
import os
import webbrowser
import warnings

# --- 1. SETUP: Try to import Folium for maps ---
try:
    import folium
    FOLIUM_INSTALLED = True
except ImportError:
    print("--- WARNING: 'folium' not installed. Map visualization will be disabled. ---")
    FOLIUM_INSTALLED = False

# Suppress warnings from interpolation, which is fine
warnings.filterwarnings('ignore', category=UserWarning)

# --- 2. THE "BRAIN": Barycentric Model Functions ---

VARIABLES_TO_PREDICT = [
    'temperature', 
    'pressure',
    'wind_direction', 
    'wind_speed', 
    'visibility', 
    'ceiling_ft_asl'
]

def get_virtual_weather_for_station(station_name, query_timestamp, query_alt, df_master_station):
    """
    Performs a 2-step interpolation (altitude, then time) for one station.
    This is our "Z" and "T" axis solver.
    """
    final_predictions = {}
    for var in VARIABLES_TO_PREDICT:
        df_var = df_master_station.dropna(subset=[var, 'timestamp', 'altitude_ft_asl'])
        if len(df_var) < 4: 
            final_predictions[var] = np.nan
            continue
        
        # Step A: Altitude Interpolation
        time_groups = df_var.groupby('timestamp')
        alt_interp_data = []
        for time, group in time_groups:
            group = group.sort_values(by='altitude_ft_asl').drop_duplicates(subset=['altitude_ft_asl'])
            if len(group) < 2: continue
            alt_interp_func = interp1d(
                group['altitude_ft_asl'], group[var], kind='linear', fill_value="extrapolate"
            )
            alt_interp_data.append({'timestamp': time, 'value': alt_interp_func(query_alt)})
            
        # Step B: Time Interpolation
        if len(alt_interp_data) < 2:
            final_predictions[var] = np.nan
            continue
            
        df_time_interp = pd.DataFrame(alt_interp_data).sort_values(by='timestamp').drop_duplicates(subset=['timestamp'])
        time_interp_func = interp1d(
            df_time_interp['timestamp'], df_time_interp['value'], kind='linear', fill_value="extrapolate"
        )
        final_predictions[var] = time_interp_func(query_timestamp)
        
    return final_predictions

def calculate_barycentric_weights(p, a, b, c):
    """
    Calculates the 3 barycentric weights of point p inside triangle (a, b, c).
    Uses the 2D vector coordinates (lon, lat).
    """
    v0 = b - a
    v1 = c - a
    v2 = p - a
    
    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)
    
    denom = d00 * d11 - d01 * d01
    if denom == 0: # Avoid division by zero for collinear points
        return None, None, None

    wB = (d11 * d20 - d01 * d21) / denom
    wC = (d00 * d21 - d01 * d20) / denom
    wA = 1.0 - wB - wC
    
    return wA, wB, wC

def predict_barycentric(query_date, query_time, query_lat, query_lon, query_alt, df_master, df_aero):
    """
    Main prediction function using proper Barycentric Interpolation
    as requested by Andreas.
    """
    try:
        query_dt = pd.to_datetime(f"{query_date} {query_time}")
        query_timestamp = query_dt.value // 10**9
    except Exception as e:
        print(f"  > Error: Invalid date/time: {e}")
        return None, None
        
    if not (-90 <= query_lat <= 90) or not (-180 <= query_lon <= 180):
        print(f"  > Error: Invalid Coordinates.")
        return None, None
        
    # --- Step 1: Create the "Virtual" 2D Weather Map ---
    # We get the "virtual weather" at the query time/alt for ALL stations
    print("  > Creating virtual 2D weather map for this time/altitude...")
    virtual_weather_reports = []
    
    # We must use the original df_aero to get all stations
    for station_name in df_aero['station'].unique():
        df_master_station = df_master[df_master['station'] == station_name]
        if len(df_master_station) == 0:
            continue
            
        virtual_weather = get_virtual_weather_for_station(
            station_name, query_timestamp, query_alt, df_master_station
        )
        
        # We need the station's coordinates
        station_info = df_aero[df_aero['station'] == station_name].iloc[0]
        
        virtual_weather_reports.append({
            'name': station_name,
            'lat': station_info['latitude'],
            'lon': station_info['longitude'],
            'alt': station_info['elevation_ft'],
            'weather': virtual_weather
        })

    # --- Step 2: Build Triangulation ---
    # Filter out stations that had no valid data to create the virtual report
    valid_stations = [s for s in virtual_weather_reports if not all(np.isnan(v) for v in s['weather'].values())]
    
    if len(valid_stations) < 3:
        print("  > Error: Not enough data (need at least 3 stations) to build triangulation.")
        return None, None

    # We MUST use (lon, lat) for 2D cartesian-like coordinates
    points = np.array([[s['lon'], s['lat']] for s in valid_stations])
    query_point = np.array([query_lon, query_lat])
    
    print("  > Building Delaunay triangulation...")
    try:
        tri = Delaunay(points)
    except Exception as e:
        print(f"  > Error during triangulation: {e}")
        return None, None

    # --- Step 3: Find Triangle and Interpolate ---
    simplex_index = tri.find_simplex(query_point)
    
    if simplex_index == -1:
        # Point is OUTSIDE the triangulation mesh. We cannot interpolate.
        # We must extrapolate. We will use the 3 nearest stations as a fallback.
        print("  > Warning: Point is outside the data mesh. Extrapolating...")
        
        # Calculate distances for all valid stations
        for s in valid_stations:
            s['distance'] = haversine((query_lat, query_lon), (s['lat'], s['lon']))
        
        # Sort by distance and take the 3 nearest
        sorted_stations = sorted(valid_stations, key=lambda s: s['distance'])
        if len(sorted_stations) < 3:
            print("  > Error: Not enough stations for extrapolation fallback.")
            return None, None
            
        sA, sB, sC = sorted_stations[0], sorted_stations[1], sorted_stations[2]
        
    else:
        # Point is INSIDE the mesh. This is the normal case.
        print("  > Point found in triangle. Interpolating...")
        # Get the indices of the 3 vertices of the triangle
        v0_idx, v1_idx, v2_idx = tri.simplices[simplex_index]
        
        # Get the actual station data for those 3 vertices
        sA, sB, sC = valid_stations[v0_idx], valid_stations[v1_idx], valid_stations[v2_idx]

    # --- Step 4: Calculate Final Prediction ---
    # Get coordinates for the 3 triangle vertices
    pA = np.array([sA['lon'], sA['lat']])
    pB = np.array([sB['lon'], sB['lat']])
    pC = np.array([sC['lon'], sC['lat']])
    
    wA, wB, wC = calculate_barycentric_weights(query_point, pA, pB, pC)
    
    if wA is None:
        print("  > Error: Collinear points in triangulation. Cannot calculate weights.")
        return None, None

    final_prediction = {}
    stations_used = [sA, sB, sC]
    
    for var in VARIABLES_TO_PREDICT:
        # Get the "virtual" value for this variable from each of the 3 stations
        vA = sA['weather'].get(var, np.nan)
        vB = sB['weather'].get(var, np.nan)
        vC = sC['weather'].get(var, np.nan)
        
        # If all 3 are NaN, the result is NaN
        if np.isnan(vA) and np.isnan(vB) and np.isnan(vC):
            final_prediction[var] = np.nan
        else:
            # Replace any NaNs with 0 for the weighted sum
            # This is a standard practice, as its weight will be
            # "redistributed" if we re-normalize, but simpler to just 0*w.
            # A better method might be to re-normalize weights, but this is complex.
            # Let's use the 3 values, but only if they are valid.
            
            # Use weights to get the final value
            # Note: This assumes NaNs are handled gracefully
            # A simple weighted average (nans are ignored by nanmean)
            # is complex with barycentric weights. Let's do it manually.
            
            total_weight = 0
            total_value = 0
            if not np.isnan(vA):
                total_weight += wA
                total_value += wA * vA
            if not np.isnan(vB):
                total_weight += wB
                total_value += wB * vB
            if not np.isnan(vC):
                total_weight += wC
                total_value += wC * vC
            
            if total_weight == 0:
                final_prediction[var] = np.nan
            else:
                # We re-normalize the weights just based on the valid data
                final_prediction[var] = total_value / total_weight

    return final_prediction, stations_used


# --- 3. THE "VISUALIZER": Map Function ---

def format_prediction_html(pred_dict):
    """Helper function to create clean HTML for map popups."""
    if not pred_dict:
        return "No prediction data available."
    html = ""
    temp = pred_dict.get('temperature', np.nan)
    pres = pred_dict.get('pressure', np.nan)
    wdir = pred_dict.get('wind_direction', np.nan)
    wspd = pred_dict.get('wind_speed', np.nan)
    vis = pred_dict.get('visibility', np.nan)
    ceil = pred_dict.get('ceiling_ft_asl', np.nan)
    html += f"Temperature: {temp:.2f} C<br>"
    html += f"Pressure: {pres:.2f} hPa<br>"
    html += f"Wind Direction: {wdir:.2f} deg<br>"
    html += f"Wind Speed: {wspd:.2f} kts<br>"
    html += f"Visibility: {vis:.2f} sm<br>"
    if not np.isnan(ceil):
         html += f"Cloud Ceiling: {ceil:.0f} ft ASL<br>"
    return html

def create_and_open_map(points_for_map, stations_for_map):
    """Creates an interactive folium map and opens it in the browser."""
    if not FOLIUM_INSTALLED:
        print("\nFolium not installed. Skipping map generation.")
        return
    print("\nGenerating visual map...")
    
    canada_center = [56.1304, -106.3468]
    m = folium.Map(location=canada_center, zoom_start=4)
    canada_geojson_url = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world/master/countries/CAN.geojson'
    try:
        folium.GeoJson(
            canada_geojson_url,
            style_function=lambda x: {'color': '#3388ff', 'weight': 2, 'fillOpacity': 0.05}
        ).add_to(m)
        print("Adding Canada boundary to map...")
    except Exception as e:
        print(f"  > Warning: Could not fetch Canada boundary. Map will be generated without it. Error: {e}")

    # Add gray markers for the 3 stations used in the triangulation
    for station in stations_for_map:
        popup_html = f"<b>Station: {station['name']}</b> (Triangle Vertex)"
        popup_html += f"<br>({station['lat']:.2f}, {station['lon']:.2f}) @ {station['alt']:.0f} ft"
        popup_html += "<hr><i>(Used for Barycentric Interpolation)</i>"
        
        folium.Marker(
            location=[station['lat'], station['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='gray', icon='plane', prefix='fa')
        ).add_to(m)

    # B. Add blue markers for the main query point(s)
    route_coords = []
    for point in points_for_map:
        route_coords.append([point['lat'], point['lon']])
        popup_html = f"<b>{point['name']}</b><br>({point['lat']:.2f}, {point['lon']:.2f}) @ {point['alt']:.0f} ft<hr>"
        popup_html += format_prediction_html(point['prediction'])
            
        folium.Marker(
            location=[point['lat'], point['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # C. If it's a route (more than 1 point), draw the line
    if len(route_coords) > 1:
        folium.PolyLine(locations=route_coords, color='blue', weight=3, opacity=0.8).add_to(m)

    # D. Save and Open the map
    filepath = os.path.abspath('weather_map.html')
    m.save(filepath)
    print(f"Visual map saved to {filepath}")
    
    webbrowser.open('file://' + filepath)
    print("Opening map in your web browser...")

# --- 4. THE "FACE": Helper Functions & Main Application ---

def get_station_info(station_code, station_db):
    """Gets Lat, Lon, and Alt for a station code."""
    try:
        station_info = station_db.loc[station_code.upper()]
        return station_info['latitude'], station_info['longitude'], station_info['elevation_ft']
    except KeyError:
        return None, None, None

def interpolate_point(start, end, fraction):
    """Linearly interpolates between two values."""
    return start + (end - start) * fraction

def main():
    print("Loading and preparing weather database... (This may take a moment)")
    
    try:
        df_aero = pd.read_csv('aerodromes.csv')
        df_master = pd.read_csv('master_dataset_final.csv')
    except FileNotFoundError as e:
        print(f"--- ERROR: File not found: {e.filename} ---")
        print("Please make sure 'aerodromes.csv' and 'master_dataset_final.csv' are in the same directory.")
        print("You must run the 'STEP_1_Create_Master_Dataset.py' script first.")
        return

    print("Preprocessing data (converting time)...")
    df_master['datetime'] = pd.to_datetime(df_master['date'] + ' ' + df_master['time'])
    df_master['timestamp'] = df_master['datetime'].astype(np.int64) // 10**9
    df_master = df_master.sort_values(by=['station', 'timestamp', 'altitude_ft_asl'])
    station_db = df_aero.set_index('station')
    
    print("✅ Weather Database is Ready.")
    print("\n--- PILOT'S FLIGHT PREDICTOR (Barycentric Model) ---")
    
    while True:
        print("\nSelect Operation:")
        print("  [R] Flight Route Briefing")
        print("  [P] Point Query (Specific Time)")
        print("  [Q] Quit")
        choice = input("Your choice: ").strip().upper()

        if choice == 'Q':
            print("Safe flying. Goodbye.")
            break

        # --- [R] FLIGHT ROUTE BRIEFING ---
        elif choice == 'R':
            try:
                print("\n--- Flight Route Briefing ---")
                start_code = input("Enter Start Station (e.g., CYVR): ").upper()
                end_code = input("Enter End Station (e.g., CYYZ): ").upper()
                
                lat1, lon1, alt1 = get_station_info(start_code, station_db)
                lat2, lon2, alt2 = get_station_info(end_code, station_db)
                
                if lat1 is None or lat2 is None:
                    print("  > Error: One or both stations not found.")
                    continue
                
                q_dep_date_str = input("Enter Departure Date (YYYY-MM-DD): ")
                q_dep_time_str = input("Enter Departure Time (HH:MM:SS): ")
                q_cruise_alt = float(input("Enter Cruise Altitude (feet, e.g., 35000): "))
                q_cruise_spd = float(input("Enter Cruise Speed (knots, e.g., 450): "))
                
                dep_date = datetime.strptime(q_dep_date_str, '%Y-%m-%d').date()
                dep_time = datetime.strptime(q_dep_time_str, '%H:%M:%S').time()
                dep_dt = datetime.combine(dep_date, dep_time)
                
                total_dist_km = haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)
                total_dist_nm = total_dist_km * 0.539957
                flight_time_hours = total_dist_nm / q_cruise_spd
                
                print(f"\n...Calculating Route ({total_dist_nm:.0f} nm, approx {flight_time_hours:.1f} hours)...")

                waypoints = [
                    ("Departure", 0.0, alt1),
                    ("Climb (25%)", 0.25, q_cruise_alt),
                    ("Cruise (50%)", 0.50, q_cruise_alt),
                    ("Descent (75%)", 0.75, q_cruise_alt),
                    ("Arrival", 1.0, alt2)
                ]
                
                print(f"\n--- FLIGHT BRIEFING FOR {start_code} -> {end_code} ---")
                
                points_for_map = []
                all_stations_for_map = {} # Use a dict to store unique stations

                for (name, fraction, alt) in waypoints:
                    wp_lat = interpolate_point(lat1, lat2, fraction)
                    wp_lon = interpolate_point(lon1, lon2, fraction)
                    wp_time_dt = dep_dt + timedelta(hours=(flight_time_hours * fraction))
                    wp_date = wp_time_dt.strftime('%Y-%m-%d')
                    wp_time = wp_time_dt.strftime('%H:%M:%S')
                    
                    print(f"\n--- {name} @ {wp_time} ---")
                    print(f"  > Coords: ({wp_lat:.2f}, {wp_lon:.2f}) @ {alt:.0f} ft")
                    
                    pred, stations = predict_barycentric(wp_date, wp_time, wp_lat, wp_lon, alt, df_master, df_aero)
                    
                    points_for_map.append({
                        'lat': wp_lat, 'lon': wp_lon, 'alt': alt, 
                        'name': name, 'prediction': pred
                    })

                    if pred:
                        print(f"  > Temperature: {pred.get('temperature', np.nan):.2f} C")
                        print(f"  > Pressure: {pred.get('pressure', np.nan):.2f} hPa")
                        print(f"  > Wind Direction: {pred.get('wind_direction', np.nan):.2f} deg")
                        print(f"  > Wind Speed: {pred.get('wind_speed', np.nan):.2f} kts")
                        print(f"  > Visibility: {pred.get('visibility', np.nan):.2f} sm")
                        if not np.isnan(pred.get('ceiling_ft_asl', np.nan)):
                            print(f"  > Cloud Ceiling: {pred['ceiling_ft_asl']:.0f} ft ASL")
                    else:
                        print("  > No prediction data available for this point.")
                    
                    if stations is not None:
                        for s in stations:
                            all_stations_for_map[s['name']] = s # Add/overwrite
                
                create_and_open_map(points_for_map, list(all_stations_for_map.values()))

            except ValueError:
                print("  > Error: Invalid number for altitude/speed or incorrect date/time format.")
            except Exception as e:
                print(f"  > An unexpected error occurred: {e}")

        # --- [P] POINT QUERY (Specific Time) ---
        elif choice == 'P':
            try:
                print("\n--- Specific Point Query (Planner) ---")
                q_date = input("Enter Date (YYYY-MM-DD): ")
                q_time = input("Enter Time (HH:MM:SS, 24h): ")
                q_lat = float(input("Enter Latitude (e.g., 49.5): "))
                q_lon = float(input("Enter Longitude (e.g., -120.0): "))
                q_alt = float(input("Enter Altitude (in feet ASL): "))
                
                print("\n...Calculating...")
                
                pred, stations = predict_barycentric(q_date, q_time, q_lat, q_lon, q_alt, df_master, df_aero)
                
                if pred:
                    print(f"\n--- Predicted Weather at ({q_lat}, {q_lon}) @ {q_alt} ft ---")
                    print(f"  > Temperature: {pred.get('temperature', np.nan):.2f} C")
                    print(f"  > Pressure: {pred.get('pressure', np.nan):.2f} hPa")
                    print(f"  > Wind Direction: {pred.get('wind_direction', np.nan):.2f} deg")
                    print(f"  > Wind Speed: {pred.get('wind_speed', np.nan):.2f} kts")
                    print(f"  > Visibility: {pred.get('visibility', np.nan):.2f} sm")
                    if not np.isnan(pred.get('ceiling_ft_asl', np.nan)):
                        print(f"  > Cloud Ceiling: {pred['ceiling_ft_asl']:.0f} ft ASL")

                points_for_map = [{
                    'lat': q_lat, 'lon': q_lon, 'alt': q_alt,
                    'name': 'Query Point', 'prediction': pred
                }]
                
                stations_for_map = []
                if stations is not None:
                    stations_for_map = stations
                        
                create_and_open_map(points_for_map, stations_for_map)
            
            except ValueError:
                print("  > Error: Invalid input. Please enter numbers for lat/lon/alt.")

        else:
            print("Invalid choice. Please enter 'R', 'P', or 'Q'.")

# --- 5. RUN THE APPLICATION ---
if __name__ == "__main__":
    main()

Loading and preparing weather database... (This may take a moment)
Preprocessing data (converting time)...
✅ Weather Database is Ready.

--- PILOT'S FLIGHT PREDICTOR (Barycentric Model) ---

Select Operation:
  [R] Flight Route Briefing
  [P] Point Query (Specific Time)
  [Q] Quit

--- Specific Point Query (Planner) ---

...Calculating...
  > Creating virtual 2D weather map for this time/altitude...
  > Building Delaunay triangulation...
  > Warning: Point is outside the data mesh. Extrapolating...

--- Predicted Weather at (76.5, -100.8) @ 5000.0 ft ---
  > Temperature: -5.64 C
  > Pressure: 834.47 hPa
  > Wind Direction: 143.51 deg
  > Wind Speed: 1.79 kts
  > Visibility: 45.85 sm

Generating visual map...
  > Warning: Could not fetch Canada boundary. Map will be generated without it. Error: Extra data: line 1 column 4 (char 3)
Visual map saved to c:\Users\Deekshith J\OneDrive\Desktop\Barycentric\weather_map.html
Opening map in your web browser...

Select Operation:
  [R] Flight Rout

In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.spatial import Delaunay
from haversine import haversine, Unit
from datetime import datetime, timedelta
import os
import webbrowser
import warnings

# --- 1. SETUP: Try to import Folium for maps ---
try:
    import folium
    FOLIUM_INSTALLED = True
except ImportError:
    print("--- WARNING: 'folium' not installed. Map visualization will be disabled. ---")
    FOLIUM_INSTALLED = False

# Suppress warnings from interpolation, which is fine
warnings.filterwarnings('ignore', category=UserWarning)

# --- 2. THE "BRAIN": Barycentric Model Functions ---

VARIABLES_TO_PREDICT = [
    'temperature', 
    'pressure',
    'wind_direction', 
    'wind_speed', 
    'visibility', 
    'ceiling_ft_asl'
]

# --- THIS IS THE SPEED FIX ---
# We will only build our triangulation from the 20 nearest stations,
# not all 500+ stations in Canada. This is much faster.
K_NEAREST_FOR_MESH = 20
# -----------------------------

def get_virtual_weather_for_station(station_name, query_timestamp, query_alt, df_master_station):
    """
    Performs a 2-step interpolation (altitude, then time) for one station.
    This is our "Z" and "T" axis solver.
    """
    final_predictions = {}
    for var in VARIABLES_TO_PREDICT:
        df_var = df_master_station.dropna(subset=[var, 'timestamp', 'altitude_ft_asl'])
        if len(df_var) < 4: 
            final_predictions[var] = np.nan
            continue
        
        # Step A: Altitude Interpolation
        time_groups = df_var.groupby('timestamp')
        alt_interp_data = []
        for time, group in time_groups:
            group = group.sort_values(by='altitude_ft_asl').drop_duplicates(subset=['altitude_ft_asl'])
            if len(group) < 2: continue
            alt_interp_func = interp1d(
                group['altitude_ft_asl'], group[var], kind='linear', fill_value="extrapolate"
            )
            alt_interp_data.append({'timestamp': time, 'value': alt_interp_func(query_alt)})
            
        # Step B: Time Interpolation
        if len(alt_interp_data) < 2:
            final_predictions[var] = np.nan
            continue
            
        df_time_interp = pd.DataFrame(alt_interp_data).sort_values(by='timestamp').drop_duplicates(subset=['timestamp'])
        time_interp_func = interp1d(
            df_time_interp['timestamp'], df_time_interp['value'], kind='linear', fill_value="extrapolate"
        )
        final_predictions[var] = time_interp_func(query_timestamp)
        
    return final_predictions

def calculate_barycentric_weights(p, a, b, c):
    """
    Calculates the 3 barycentric weights of point p inside triangle (a, b, c).
    Uses the 2D vector coordinates (lon, lat).
    """
    v0 = b - a
    v1 = c - a
    v2 = p - a
    
    d00 = np.dot(v0, v0)
    d01 = np.dot(v0, v1)
    d11 = np.dot(v1, v1)
    d20 = np.dot(v2, v0)
    d21 = np.dot(v2, v1)
    
    denom = d00 * d11 - d01 * d01
    if denom == 0: # Avoid division by zero for collinear points
        return None, None, None

    wB = (d11 * d20 - d01 * d21) / denom
    wC = (d00 * d21 - d01 * d20) / denom
    wA = 1.0 - wB - wC
    
    return wA, wB, wC

def find_k_nearest_stations(query_lat, query_lon, df_aero, k=5):
    """Finds the k-nearest stations to a query point. (This is fast)"""
    query_point = (query_lat, query_lon)
    station_points = list(zip(df_aero['latitude'], df_aero['longitude']))
    df_aero['distance_km'] = [haversine(query_point, pt, unit=Unit.KILOMETERS) for pt in station_points]
    return df_aero.sort_values(by='distance_km').head(k)

def predict_barycentric(query_date, query_time, query_lat, query_lon, query_alt, df_master, df_aero):
    """
    Main prediction function using a "local" Barycentric Interpolation
    for speed and accuracy.
    """
    try:
        query_dt = pd.to_datetime(f"{query_date} {query_time}")
        query_timestamp = query_dt.value // 10**9
    except Exception as e:
        print(f"  > Error: Invalid date/time: {e}")
        return None, None
        
    if not (-90 <= query_lat <= 90) or not (-180 <= query_lon <= 180):
        print(f"  > Error: Invalid Coordinates.")
        return None, None
        
    # --- STEP 1: FIND K-NEAREST STATIONS (FAST) ---
    # This is the key optimization. We only build our mesh from nearby stations.
    print(f"  > Finding {K_NEAREST_FOR_MESH} nearest stations...")
    df_aero_copy = df_aero.copy()
    try:
        nearest_stations = find_k_nearest_stations(query_lat, query_lon, df_aero_copy, K_NEAREST_FOR_MESH)
    except ValueError as e:
        print(f"  > Error in coordinate calculation: {e}")
        return None, None
    
    if len(nearest_stations) < 3:
        print("  > Error: Not enough nearby stations to build a model.")
        return None, None
        
    # --- Step 2: Create the "Virtual" 2D Weather Map (NOW FAST) ---
    # We only run the slow interpolation for our K_NEAREST stations
    print(f"  > Creating virtual 2D weather map from {len(nearest_stations)} stations...")
    virtual_weather_reports = []
    
    for _, station in nearest_stations.iterrows():
        df_master_station = df_master[df_master['station'] == station['station']]
        if len(df_master_station) == 0:
            continue
            
        virtual_weather = get_virtual_weather_for_station(
            station['station'], query_timestamp, query_alt, df_master_station
        )
        
        virtual_weather_reports.append({
            'name': station['station'],
            'lat': station['latitude'],
            'lon': station['longitude'],
            'alt': station['elevation_ft'],
            'weather': virtual_weather
        })

    # --- Step 3: Build Triangulation ---
    # Filter out stations that had no valid data
    valid_stations = [s for s in virtual_weather_reports if not all(np.isnan(v) for v in s['weather'].values())]
    
    if len(valid_stations) < 3:
        print("  > Error: Not enough data (need at least 3 stations) to build triangulation.")
        return None, None

    # We MUST use (lon, lat) for 2D cartesian-like coordinates
    points = np.array([[s['lon'], s['lat']] for s in valid_stations])
    query_point = np.array([query_lon, query_lat])
    
    print("  > Building Delaunay triangulation...")
    try:
        tri = Delaunay(points)
    except Exception as e:
        print(f"  > Error during triangulation: {e}")
        return None, None

    # --- Step 4: Find Triangle and Interpolate ---
    simplex_index = tri.find_simplex(query_point)
    
    if simplex_index == -1:
        # Point is OUTSIDE the triangulation mesh. We cannot interpolate.
        # We must extrapolate. We will use the 3 nearest stations as a fallback.
        print("  > Warning: Point is outside the local data mesh. Extrapolating...")
        
        # Calculate distances for all valid stations
        for s in valid_stations:
            s['distance'] = haversine((query_lat, query_lon), (s['lat'], s['lon']))
        
        # Sort by distance and take the 3 nearest
        sorted_stations = sorted(valid_stations, key=lambda s: s['distance'])
        if len(sorted_stations) < 3:
            print("  > Error: Not enough stations for extrapolation fallback.")
            return None, None
            
        sA, sB, sC = sorted_stations[0], sorted_stations[1], sorted_stations[2]
        
    else:
        # Point is INSIDE the mesh. This is the normal case.
        print("  > Point found in triangle. Interpolating...")
        # Get the indices of the 3 vertices of the triangle
        v0_idx, v1_idx, v2_idx = tri.simplices[simplex_index]
        
        # Get the actual station data for those 3 vertices
        sA, sB, sC = valid_stations[v0_idx], valid_stations[v1_idx], valid_stations[v2_idx]

    # --- Step 5: Calculate Final Prediction ---
    # Get coordinates for the 3 triangle vertices
    pA = np.array([sA['lon'], sA['lat']])
    pB = np.array([sB['lon'], sB['lat']])
    pC = np.array([sC['lon'], sC['lat']])
    
    wA, wB, wC = calculate_barycentric_weights(query_point, pA, pB, pC)
    
    if wA is None:
        print("  > Error: Collinear points in triangulation. Cannot calculate weights.")
        return None, None

    final_prediction = {}
    stations_used = [sA, sB, sC]
    
    for var in VARIABLES_TO_PREDICT:
        # Get the "virtual" value for this variable from each of the 3 stations
        vA = sA['weather'].get(var, np.nan)
        vB = sB['weather'].get(var, np.nan)
        vC = sC['weather'].get(var, np.nan)
        
        if np.isnan(vA) and np.isnan(vB) and np.isnan(vC):
            final_prediction[var] = np.nan
        else:
            total_weight = 0
            total_value = 0
            if not np.isnan(vA):
                total_weight += wA
                total_value += wA * vA
            if not np.isnan(vB):
                total_weight += wB
                total_value += wB * vB
            if not np.isnan(vC):
                total_weight += wC
                total_value += wC * vC
            
            if total_weight == 0:
                final_prediction[var] = np.nan
            else:
                # We re-normalize the weights just based on the valid data
                final_prediction[var] = total_value / total_weight

    return final_prediction, stations_used


# --- 3. THE "VISUALIZER": Map Function ---

def format_prediction_html(pred_dict):
    """Helper function to create clean HTML for map popups."""
    if not pred_dict:
        return "No prediction data available."
    html = ""
    temp = pred_dict.get('temperature', np.nan)
    pres = pred_dict.get('pressure', np.nan)
    wdir = pred_dict.get('wind_direction', np.nan)
    wspd = pred_dict.get('wind_speed', np.nan)
    vis = pred_dict.get('visibility', np.nan)
    ceil = pred_dict.get('ceiling_ft_asl', np.nan)
    html += f"Temperature: {temp:.2f} C<br>"
    html += f"Pressure: {pres:.2f} hPa<br>"
    html += f"Wind Direction: {wdir:.2f} deg<br>"
    html += f"Wind Speed: {wspd:.2f} kts<br>"
    html += f"Visibility: {vis:.2f} sm<br>"
    if not np.isnan(ceil):
         html += f"Cloud Ceiling: {ceil:.0f} ft ASL<br>"
    return html

def create_and_open_map(points_for_map, stations_for_map):
    """Creates an interactive folium map and opens it in the browser."""
    if not FOLIUM_INSTALLED:
        print("\nFolium not installed. Skipping map generation.")
        return
    print("\nGenerating visual map...")
    
    canada_center = [56.1304, -106.3468]
    m = folium.Map(location=canada_center, zoom_start=4)
    canada_geojson_url = 'https://raw.githubusercontent.com/datasets/geo-boundaries-world/master/countries/CAN.geojson'
    try:
        folium.GeoJson(
            canada_geojson_url,
            style_function=lambda x: {'color': '#3388ff', 'weight': 2, 'fillOpacity': 0.05}
        ).add_to(m)
        print("Adding Canada boundary to map...")
    except Exception as e:
        print(f"  > Warning: Could not fetch Canada boundary. Map will be generated without it. Error: {e}")

    # Add gray markers for the 3 stations used in the triangulation
    for station in stations_for_map:
        popup_html = f"<b>Station: {station['name']}</b> (Triangle Vertex)"
        popup_html += f"<br>({station['lat']:.2f}, {station['lon']:.2f}) @ {station['alt']:.0f} ft"
        popup_html += "<hr><i>(Used for Barycentric Interpolation)</i>"
        
        folium.Marker(
            location=[station['lat'], station['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='gray', icon='plane', prefix='fa')
        ).add_to(m)

    # B. Add blue markers for the main query point(s)
    route_coords = []
    for point in points_for_map:
        route_coords.append([point['lat'], point['lon']])
        popup_html = f"<b>{point['name']}</b><br>({point['lat']:.2f}, {point['lon']:.2f}) @ {point['alt']:.0f} ft<hr>"
        popup_html += format_prediction_html(point['prediction'])
            
        folium.Marker(
            location=[point['lat'], point['lon']],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

    # C. If it's a route (more than 1 point), draw the line
    if len(route_coords) > 1:
        folium.PolyLine(locations=route_coords, color='blue', weight=3, opacity=0.8).add_to(m)

    # D. Save and Open the map
    filepath = os.path.abspath('weather_map.html')
    m.save(filepath)
    print(f"Visual map saved to {filepath}")
    
    webbrowser.open('file://' + filepath)
    print("Opening map in your web browser...")

# --- 4. THE "FACE": Helper Functions & Main Application ---

def get_station_info(station_code, station_db):
    """Gets Lat, Lon, and Alt for a station code."""
    try:
        station_info = station_db.loc[station_code.upper()]
        return station_info['latitude'], station_info['longitude'], station_info['elevation_ft']
    except KeyError:
        return None, None, None

def interpolate_point(start, end, fraction):
    """Linearly interpolates between two values."""
    return start + (end - start) * fraction

def main():
    print("Loading and preparing weather database... (This may take a moment)")
    
    try:
        df_aero = pd.read_csv('aerodromes.csv')
        df_master = pd.read_csv('master_dataset_final.csv')
    except FileNotFoundError as e:
        print(f"--- ERROR: File not found: {e.filename} ---")
        print("Please make sure 'aerodromes.csv' and 'master_dataset_final.csv' are in the same directory.")
        print("You must run the 'STEP_1_Create_Master_Dataset.py' script first.")
        return

    print("Preprocessing data (converting time)...")
    df_master['datetime'] = pd.to_datetime(df_master['date'] + ' ' + df_master['time'])
    df_master['timestamp'] = df_master['datetime'].astype(np.int64) // 10**9
    df_master = df_master.sort_values(by=['station', 'timestamp', 'altitude_ft_asl'])
    station_db = df_aero.set_index('station')
    
    print("✅ Weather Database is Ready.")
    print("\n--- PILOT'S FLIGHT PREDICTOR (Barycentric Model v2 - FAST) ---")
    
    while True:
        print("\nSelect Operation:")
        print("  [R] Flight Route Briefing")
        print("  [P] Point Query (Specific Time)")
        print("  [Q] Quit")
        choice = input("Your choice: ").strip().upper()

        if choice == 'Q':
            print("Safe flying. Goodbye.")
            break

        # --- [R] FLIGHT ROUTE BRIEFING ---
        elif choice == 'R':
            try:
                print("\n--- Flight Route Briefing ---")
                start_code = input("Enter Start Station (e.g., CYVR): ").upper()
                end_code = input("Enter End Station (e.g., CYYZ): ").upper()
                
                lat1, lon1, alt1 = get_station_info(start_code, station_db)
                lat2, lon2, alt2 = get_station_info(end_code, station_db)
                
                if lat1 is None or lat2 is None:
                    print("  > Error: One or both stations not found.")
                    continue
                
                q_dep_date_str = input("Enter Departure Date (YYYY-MM-DD): ")
                q_dep_time_str = input("Enter Departure Time (HH:MM:SS): ")
                q_cruise_alt = float(input("Enter Cruise Altitude (feet, e.g., 35000): "))
                q_cruise_spd = float(input("Enter Cruise Speed (knots, e.g., 450): "))
                
                dep_date = datetime.strptime(q_dep_date_str, '%Y-%m-%d').date()
                dep_time = datetime.strptime(q_dep_time_str, '%H:%M:%S').time()
                dep_dt = datetime.combine(dep_date, dep_time)
                
                total_dist_km = haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)
                total_dist_nm = total_dist_km * 0.539957
                flight_time_hours = total_dist_nm / q_cruise_spd
                
                print(f"\n...Calculating Route ({total_dist_nm:.0f} nm, approx {flight_time_hours:.1f} hours)...")

                waypoints = [
                    ("Departure", 0.0, alt1),
                    ("Climb (25%)", 0.25, q_cruise_alt),
                    ("Cruise (50%)", 0.50, q_cruise_alt),
                    ("Descent (75%)", 0.75, q_cruise_alt),
                    ("Arrival", 1.0, alt2)
                ]
                
                print(f"\n--- FLIGHT BRIEFING FOR {start_code} -> {end_code} ---")
                
                points_for_map = []
                all_stations_for_map = {} # Use a dict to store unique stations

                for (name, fraction, alt) in waypoints:
                    wp_lat = interpolate_point(lat1, lat2, fraction)
                    wp_lon = interpolate_point(lon1, lon2, fraction)
                    wp_time_dt = dep_dt + timedelta(hours=(flight_time_hours * fraction))
                    wp_date = wp_time_dt.strftime('%Y-%m-%d')
                    wp_time = wp_time_dt.strftime('%H:%M:%S')
                    
                    print(f"\n--- {name} @ {wp_time} ---")
                    print(f"  > Coords: ({wp_lat:.2f}, {wp_lon:.2f}) @ {alt:.0f} ft")
                    
                    pred, stations = predict_barycentric(wp_date, wp_time, wp_lat, wp_lon, alt, df_master, df_aero)
                    
                    points_for_map.append({
                        'lat': wp_lat, 'lon': wp_lon, 'alt': alt, 
                        'name': name, 'prediction': pred
                    })

                    if pred:
                        print(f"  > Temperature: {pred.get('temperature', np.nan):.2f} C")
                        print(f"  > Pressure: {pred.get('pressure', np.nan):.2f} hPa")
                        print(f"  > Wind Direction: {pred.get('wind_direction', np.nan):.2f} deg")
                        print(f"  > Wind Speed: {pred.get('wind_speed', np.nan):.2f} kts")
                        print(f"  > Visibility: {pred.get('visibility', np.nan):.2f} sm")
                        if not np.isnan(pred.get('ceiling_ft_asl', np.nan)):
                            print(f"  > Cloud Ceiling: {pred['ceiling_ft_asl']:.0f} ft ASL")
                    else:
                        print("  > No prediction data available for this point.")
                    
                    if stations is not None:
                        for s in stations:
                            all_stations_for_map[s['name']] = s # Add/overwrite
                
                create_and_open_map(points_for_map, list(all_stations_for_map.values()))

            except ValueError:
                print("  > Error: Invalid number for altitude/speed or incorrect date/time format.")
            except Exception as e:
                print(f"  > An unexpected error occurred: {e}")

        # --- [P] POINT QUERY (Specific Time) ---
        elif choice == 'P':
            try:
                print("\n--- Specific Point Query (Planner) ---")
                q_date = input("Enter Date (YYYY-MM-DD): ")
                q_time = input("Enter Time (HH:MM:SS, 24h): ")
                q_lat = float(input("Enter Latitude (e.g., 49.5): "))
                q_lon = float(input("Enter Longitude (e.g., -120.0): "))
                q_alt = float(input("Enter Altitude (in feet ASL): "))
                
                print("\n...Calculating...")
                
                pred, stations = predict_barycentric(q_date, q_time, q_lat, q_lon, q_alt, df_master, df_aero)
                
                if pred:
                    print(f"\n--- Predicted Weather at ({q_lat}, {q_lon}) @ {q_alt} ft ---")
                    print(f"  > Temperature: {pred.get('temperature', np.nan):.2f} C")
                    print(f"  > Pressure: {pred.get('pressure', np.nan):.2f} hPa")
                    print(f"  > Wind Direction: {pred.get('wind_direction', np.nan):.2f} deg")
                    print(f"  > Wind Speed: {pred.get('wind_speed', np.nan):.2f} kts")
                    print(f"  > Visibility: {pred.get('visibility', np.nan):.2f} sm")
                    if not np.isnan(pred.get('ceiling_ft_asl', np.nan)):
                        print(f"  > Cloud Ceiling: {pred['ceiling_ft_asl']:.0f} ft ASL")

                points_for_map = [{
                    'lat': q_lat, 'lon': q_lon, 'alt': q_alt,
                    'name': 'Query Point', 'prediction': pred
                }]
                
                stations_for_map = []
                if stations is not None:
                    stations_for_map = stations
                        
                create_and_open_map(points_for_map, stations_for_map)
            
            except ValueError:
                print("  > Error: Invalid input. Please enter numbers for lat/lon/alt.")

        else:
            print("Invalid choice. Please enter 'R', 'P', or 'Q'.")

# --- 5. RUN THE APPLICATION ---
if __name__ == "__main__":
    main()

Loading and preparing weather database... (This may take a moment)
Preprocessing data (converting time)...
✅ Weather Database is Ready.

--- PILOT'S FLIGHT PREDICTOR (Barycentric Model v2 - FAST) ---

Select Operation:
  [R] Flight Route Briefing
  [P] Point Query (Specific Time)
  [Q] Quit

--- Specific Point Query (Planner) ---

...Calculating...
  > Finding 20 nearest stations...
  > Creating virtual 2D weather map from 20 stations...
  > Building Delaunay triangulation...
  > Point found in triangle. Interpolating...

--- Predicted Weather at (56.4, -100.2) @ 5000.0 ft ---
  > Temperature: -5.74 C
  > Pressure: 832.27 hPa
  > Wind Direction: 232.59 deg
  > Wind Speed: 2.83 kts
  > Visibility: 18.01 sm

Generating visual map...
  > Warning: Could not fetch Canada boundary. Map will be generated without it. Error: Extra data: line 1 column 4 (char 3)
Visual map saved to c:\Users\Deekshith J\OneDrive\Desktop\Barycentric\weather_map.html
Opening map in your web browser...

Select Opera